In [18]:
##ossec-hids中的rule配置和测试数据可用于日志分析

from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET
import xmltodict, json
import pandas as pd
import collections
import re
from pandas.io.json import json_normalize

pd.options.display.max_colwidth = 1000
filepath = r"F:\open-source-check\ossec-xml"
EXCLUDE_FILES=["decode.xml",
              "local_rules.xml"]

In [19]:
def getAllXMLFiles(destPath):
    filelist = []
    for f in listdir(destPath):
        if not isfile(join(destPath, f)):
            continue
            
        if not f.endswith("rules_en.xml") \
            and not f.endswith("rules.xml"):
                continue
        if f in EXCLUDE_FILES:
            continue
            
        filelist.append(f)
    
    return filelist

"""
filelist = getAllXMLFiles(filepath)
for file in filelist:
    print(file)
"""

'\nfilelist = getAllXMLFiles(filepath)\nfor file in filelist:\n    print(file)\n'

In [20]:
def parseXMLToPd(file): 
    try:
        with open(file, "r", encoding='utf-8') as fd:
            #https://github.com/martinblech/xmltodict/issues/2
            data = xmltodict.parse('<root>{0}</root>'.format(fd.read()))['root']

            #get all the vars
            df_vars = pd.DataFrame()
            if "var" in data:
                try:
                    df_vars = pd.DataFrame(json_normalize(data["var"]))
                except Exception as e:
                    print(e)
                    raise e
            
            #replace all the vars in jsonStr
            jsonData = json.dumps(data["group"], indent=4)
            for index, row in df_vars.iterrows():
                key, value = row["@name"], row["#text"]
                #for Unrecognized escape sequence
                value = re.sub(r"([\\\/])", r"\\\1", value)
                jsonData = jsonData.replace('$'+key, value)
                
            #print(jsonData)  
            return pd.read_json(jsonData)
    except Exception as e:
        print(file)
        raise e
    
    return None
    
#print(parseXMLToPd(join(filepath, "web_appsec_rules.xml")).head())
#print(parseXMLToPd(join(filepath, "syslog_rules.xml")).head())
#print(parseXMLToPd(join(filepath, "mcafee_av_rules.xml")).head())

In [21]:
filelist = getAllXMLFiles(filepath)
df = pd.DataFrame()
for file in filelist:
    df_tmp = parseXMLToPd(join(filepath, file))
    if df_tmp is not None:
        df = df.append(df_tmp, ignore_index=True)
print(df.shape)
df.to_csv("ossec_rules.csv", sep=',', encoding='utf-8', index=False)     

(1063, 2)


In [28]:
df_keep = pd.DataFrame()
df_bak = pd.DataFrame()
for index, row in df.iterrows():
    group_name = row["@name"]
    rules = row["rule"]
    df_tmp = None
    try:
        df_tmp = pd.DataFrame(json_normalize(rules))
        df_tmp["group_name"] = group_name
        df_bak = df_bak.append(df_tmp, ignore_index=True)
        keep_columns = ["group_name", "body"]
        if "description" in df_tmp.columns:
            keep_columns.append("description")
            
        join_columns = []
        if "match" in df_tmp.columns:
            join_columns.append("match")
        elif "regex" in df_tmp.columns:
            join_columns.append("regex")
            if "regex.#text" in df_tmp.columns:
                join_columns.append("regex.#text")
        else:
            continue
        df_tmp["body"] = df_tmp[join_columns].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
        #df_tmp["body"] = df_tmp[keep_columns].stack().values

        df_keep = df_keep.append(df_tmp[keep_columns], ignore_index=True)
    except Exception as e:
        print(rules)
        print(df_tmp)
        print(e)
        
print(df_keep.head(1))
print(df_keep.columns)
print(df_keep.shape)
print(df_bak.columns)
print(df_bak.shape)
df_keep.to_csv("ossec_rules_keep.csv", sep=',', encoding='utf-8', index=False)   
#print(df_keep[df_keep["@id"] == "7300"][["@id", "@level", "decoded_as", "description"]])

  group_name      body                     description
0    apache,  ^[error]  Apache error messages grouped.
Index(['group_name', 'body', 'description'], dtype='object')
(661, 3)
Index(['@frequency', '@id', '@ignore', '@level', '@maxsize', '@noalert',
       '@timeframe', 'action', 'category', 'check_diff', 'check_if_ignored',
       'compiled_rule', 'decoded_as', 'description', 'different_url',
       'extra_data', 'group', 'group_name', 'hostname', 'id', 'if_fts',
       'if_group', 'if_matched_group', 'if_matched_sid', 'if_sid', 'ignore',
       'info', 'info.#text', 'info.@type', 'match', 'options', 'program_name',
       'regex', 'regex.#text', 'regex.@offet', 'same_id', 'same_source_ip',
       'same_user', 'status', 'time', 'url', 'user', 'weekday'],
      dtype='object')
(1197, 43)


In [ ]:
print